###### 【問題1】学習と推定
READMEを参考に上記実装を動かしてください。

Epoch=3 Iteration=100で学習した 

###### 【問題2】論文と実装の対応
コードリーディングを行ってください。
まず、Faster R-CNN[1]において重要だと考えた部分を列挙してください。そして、それに対応するコードを見つけてください。
（例）
RPNを実現しているコードはどこか
RoIプーリングを実現しているコードはどこか

RPN

In [ ]:
1.Modelを作る  
    #config fileから学習したRPNのパラメータを読み込みモデルを構築(Resnet)
    shared_layers = nn.nn_base(img_input, trainable=True)
    # define the RPN, built on the base layers
    num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
    # RPNのパラメータを与えRPNを作成
    rpn = nn.rpn(shared_layers, num_anchors)
    model_rpn = Model(img_input, rpn[:2])
    # Loss関数==> クラス分類：losses.rpn_loss_cls BoundingBox位置の回帰 :losses.rpn_loss_regr
    model_rpn.compile(optimizer=Adam(lr=1e-4), loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
    
    #モデル
    def rpn(base_layers,num_anchors):
        x = Convolution2D(512, (3, 3), padding='same', activation='relu', kernel_initializer='normal', name='rpn_conv1')(base_layers)
        x_class = Convolution2D(num_anchors, (1, 1), activation='sigmoid', kernel_initializer='uniform', name='rpn_out_class')(x)
        x_regr = Convolution2D(num_anchors * 4, (1, 1), activation='linear', kernel_initializer='zero', name='rpn_out_regress')(x)
        return [x_class, x_regr, base_layers]
    
    
2.学習  
    #Modelインスタンスを作成
    model_rpn, model_classifier, model_all = faster_rcnn.get_model(C, classes_count)
    
    #Batch分ループを回し学習
    #X=input image, Y=[Y_RPN_Class, Y_RPN_POS(x,y,h,w), Image(arugment関数へ入力されたoriginalイメージ）
    X, Y, img_data = next(data_gen_train)
    #RPNの学習
    loss_rpn = model_rpn.train_on_batch(X, Y)


3.予測  
    #Modelインスタンスを作成
    model_rpn, model_classifier, model_classifier_only = get_models(C)
    #領域の予測
    [Y1, Y2, F] = model_rpn.predict(X)
    R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)
    

Classifier 

In [ ]:
1.Modelを作る
    classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=True)
    model_classifier = Model([img_input, roi_input], classifier)
    model_classifier.compile(optimizer=Adam(lr=1e-4), loss=[losses.class_loss_cls, losses.class_loss_regr(len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})

    #モデル 内部にRoiPoolingを持つ
    def classifier(base_layers, input_rois, num_rois, nb_classes = 21, trainable=False):
        pooling_regions = 14
        input_shape = (num_rois,14,14,1024)
        out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([base_layers, input_rois])
        out = classifier_layers(out_roi_pool, input_shape=input_shape, trainable=True)
        out = TimeDistributed(Flatten())(out)
        out_class = TimeDistributed(Dense(nb_classes, activation='softmax', kernel_initializer='zero'), name='dense_class_{}'.format(nb_classes))(out)
        # note: no regression target for bg class
        out_regr = TimeDistributed(Dense(4 * (nb_classes-1), activation='linear', kernel_initializer='zero'), name='dense_regress_{}'.format(nb_classes))(out)
        return [out_class, out_regr]    
    
2.学習 
    loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])


3.予測
#RPNで領域候補を抽出
[Y1, Y2, F] = model_rpn.predict(X)
R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)

for jk in range(R.shape[0]//C.num_rois + 1):
    ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
    if ROIs.shape[1] == 0:
        break

    if jk == R.shape[0]//C.num_rois:
        #pad R
        curr_shape = ROIs.shape
        target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
        ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
        ROIs_padded[:, :curr_shape[1], :] = ROIs
        ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
        ROIs = ROIs_padded
#Classifierでクラス分類と位置を取得
    [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

ROI pooling

In [ ]:
#Classifierの内部にRoIPoolingを持つ
def classifier(base_layers, input_rois, num_rois, nb_classes = 21, trainable=False):

    pooling_regions = 14
    input_shape = (num_rois,14,14,1024)
    out_roi_pool = RoiPoolingConv(pooling_regions, num_rois)([base_layers, input_rois])
    out = classifier_layers(out_roi_pool, input_shape=input_shape, trainable=True)

##### シンプソンズのデータセットをFaster R-CNN以外の手法で学習・推定を行います。YOLOv3[2]のKeras実装を使います。

###### 【問題3】学習済みの重みによる推定
学習済みの重みを使い推定を行う方法がREADME.mdのQuick Startに記載されています。
まずはこの通りにして各自何かしらの画像や動画に対して検出を行ってください。
出力結果を課題の一部として提出してください。

<img src="result_q3.png">

学習していないデータを入力したため、正しく分類できていない。

###### 【問題4】学習のためのファイルを作成
新しいデータ（シンプソンズデータセット）を学習します。README.mdのTrainingを読み、シンプソンズデータセットを学習するために必要なファイルを作成してください。
アノテーションファイルの形式がSprint18で扱った実装のものとは異なっているので、変換する必要があります。

In [24]:
import pandas as pd
import numpy as np

In [120]:
file = pd.read_csv("annotation.txt")
file.head()

,Filename,x_min,y_min,x_max,y_max,class_id
0,simpsons_dataset/abraham_grampa_simpson/pic_00...,57,72,52,72,abraham_grampa_simpson
1,simpsons_dataset/abraham_grampa_simpson/pic_00...,80,31,337,354,abraham_grampa_simpson
2,simpsons_dataset/abraham_grampa_simpson/pic_00...,128,48,285,407,abraham_grampa_simpson
3,simpsons_dataset/abraham_grampa_simpson/pic_00...,72,126,158,275,abraham_grampa_simpson
4,simpsons_dataset/abraham_grampa_simpson/pic_00...,123,61,294,416,abraham_grampa_simpson


In [121]:
class_name = np.unique(file["class_id"])

In [25]:
class_mapping = {label:idx for idx, label in enumerate(np.unique(file["class_id"]))}

In [26]:
print(class_mapping)

{'abraham_grampa_simpson': 0, 'apu_nahasapeemapetilon': 1, 'bart_simpson': 2, 'charles_montgomery_burns': 3, 'chief_wiggum': 4, 'comic_book_guy': 5, 'edna_krabappel': 6, 'homer_simpson': 7, 'kent_brockman': 8, 'krusty_the_clown': 9, 'lisa_simpson': 10, 'marge_simpson': 11, 'milhouse_van_houten': 12, 'moe_szyslak': 13, 'ned_flanders': 14, 'nelson_muntz': 15, 'principal_skinner': 16, 'sideshow_bob': 17}


In [27]:
 file["class_id"] =  file["class_id"].map(class_mapping)

In [92]:
file = file.reset_index(drop=True)
filename = file["Filename"].astype(str)
x_min = file["x_min"].astype(str)
y_min = file["y_min"].astype(str)
x_max = file["x_max"].astype(str)
y_max = file["y_max"].astype(str)
class_id = file["class_id"].astype(str)

In [112]:
tmp = filename + str(" ") + x_min + str(",") + y_min + str(",") + x_max + str(",") + y_max + str(",") + class_id

7889

In [119]:
tmp.to_csv("train.txt", index=False, sep='"')

In [128]:
cls = pd.DataFrame(class_name)
cls.to_csv("simpsons_class.txt", index=False, header=False, sep='"')

###### 【問題5】学習
問題2で作成したファイルを使用して学習してください。実行環境で学習に時間がかかる場合は、学習が行えることを確認するのみで終えて構いません。

<img src="yolo3_exec_result.png">

###### 【問題6】（アドバンス課題）論文と実装の対応
コードリーディングを行ってください。
まず、YOLOv3[2]の論文において重要だと考えた部分を列挙してください。そして、それに対応するコードを見つけてください。